In [5]:
# Set the base URL
BASE_URL="http://localhost:8080"

In [10]:
import json
import acsets, petris
from requests import get, put, post, delete
from graphviz import Source

In [20]:
MODEL_ID = 1787
opos = get(f"{BASE_URL}/models/{MODEL_ID}/named_opos").json()
new_opos = []
for ent in opos: 
    new_opos.append(ent.replace("d", ""))

print(new_opos)

['St', 'Et', 'It', 'Rt', 'Dt', 'Nt']


In [8]:
opis = get(f"{BASE_URL}/models/{MODEL_ID}/named_opis").json()
print(opis)

['y', 't', 'N0', 'alpha', 'beta', 'gamma', 'epsilon', 'mu']


In [23]:
seird = petris.Petri()
tuple(new_opos)= seird.add_species(len(new_opos))


SyntaxError: cannot assign to function call (851457303.py, line 2)

In [26]:
sir = petris.Petri()
s,i,r = sir.add_species(3)
trans = petris.Transition
inf,rec = sir.add_parts(trans, 2)
sir.set_subpart(0, attr_name, "inf")

serialized = sir.write_json()
deserialized = petris.Petri.read_json(petris.SchPetri, serialized)
reserialized = deserialized.write_json()

print(serialized)
print(reserialized)

{"S": [{}, {}, {}], "T": [{}, {}], "I": [], "O": []}
{"S": [{}, {}, {}], "T": [{}, {}], "I": [], "O": []}
